In [58]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
from tensorboardX import SummaryWriter

try:
    from src.earlystopping import EarlyStopping
except:
    import sys, os
    sys.path.append(os.path.realpath('../'))
    from src.earlystopping import EarlyStopping
    
from src.sample import Sampler
from src.metric import accuracy, roc_auc_compute_fn
# from deepgcn.utils import load_data, accuracy
# from deepgcn.models import GCN

from src.metric import accuracy
from src.utils import load_citation, load_reddit_data
from src.models import *

# Training settings
parser = argparse.ArgumentParser()
opt = parser.parse_args([])

opt.debug = True
opt.dataset = "cora"
opt.datapath = "data/"
parser.add_argument('--dataset', default="cora", help="The data set")

opt.seed = 42
opt.epochs = 50
opt.lr = 0.02
opt.weight_decay = 5e-4
opt.hidden = 128
opt.dropout = 0
opt.nhiddenlayer=1
opt.early_stopping = 0
opt.sampling_percent = 1.0

opt.normalization = "AugNormAdj"
opt.nbaseblocklayer = 6
opt.type = "inceptiongcn"

opt.inputlayer = 'gcn'
opt.outputlayer = 'gcn'

opt.lradjust = False
opt.no_cuda = False
opt.mixmode = False
opt.aggrmethod = "default"
opt.withloop = False
opt.no_tensorboard = False
opt.withbn = False
opt.fastmode = "default"
opt.warm_start = ""

opt.no_cuda = False
opt.task_type = "full"

In [59]:
if opt.debug:
    print(opt)
# pre setting
opt.cuda = not opt.no_cuda and torch.cuda.is_available()
opt.mixmode = opt.no_cuda and opt.mixmode and torch.cuda.is_available()
if opt.aggrmethod == "default":
    if opt.type == "resgcn":
        opt.aggrmethod = "add"
    else:
        opt.aggrmethod = "concat"
if opt.fastmode and opt.early_stopping > 0:
    opt.early_stopping = 0
    print("In the fast mode, early_stopping is not valid option. Setting early_stopping = 0.")
if opt.type == "mutigcn":
    print("For the multi-layer gcn model, the aggrmethod is fixed to nores and nhiddenlayers = 1.")
    opt.nhiddenlayer = 1
    opt.aggrmethod = "nores"

# random seed setting
np.random.seed(opt.seed)
torch.manual_seed(opt.seed)
if opt.cuda or opt.mixmode:
    torch.cuda.manual_seed(opt.seed)

# should we need fix random seed here?
sampler = Sampler(opt.dataset, opt.datapath, opt.task_type)

Namespace(aggrmethod='default', datapath='data/', dataset='cora', debug=True, dropout=0, early_stopping=0, epochs=50, fastmode='default', hidden=128, inputlayer='gcn', lr=0.02, lradjust=False, mixmode=False, nbaseblocklayer=6, nhiddenlayer=1, no_cuda=False, no_tensorboard=False, normalization='AugNormAdj', outputlayer='gcn', sampling_percent=1.0, seed=42, task_type='full', type='inceptiongcn', warm_start='', weight_decay=0.0005, withbn=False, withloop=False)
Load full supervised task.


In [70]:
# get labels and indexes
labels, idx_train, idx_val, idx_test = sampler.get_label_and_idxes(opt.cuda)
nfeat = sampler.nfeat
nclass = sampler.nclass
print("nclass: %d\tnfeat: %d" % (nclass, nfeat))

# The model
model = GCNModel(nfeat=nfeat,
                 nhid=opt.hidden,
                 nclass=nclass,
                 nhidlayer=opt.nhiddenlayer,
                 dropout=opt.dropout,
                 baseblock=opt.type,
                 inputlayer=opt.inputlayer,
                 outputlayer=opt.outputlayer,
                 nbaselayer=opt.nbaseblocklayer,
                 activation=F.relu,
                 withbn=opt.withbn,
                 withloop=opt.withloop,
                 aggrmethod=opt.aggrmethod,
                 mixmode=opt.mixmode)

optimizer = optim.Adam(model.parameters(),
                       lr=opt.lr, weight_decay=opt.weight_decay)

# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, factor=0.618)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200, 300, 400, 500, 600, 700], gamma=0.5)
# convert to cuda
if opt.cuda:
    model.cuda()

# For the mix mode, lables and indexes are in cuda. 
if opt.cuda or opt.mixmode:
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

if opt.warm_start is not None and opt.warm_start != "":
    early_stopping = EarlyStopping(fname=opt.warm_start, verbose=False)
    print("Restore checkpoint from %s" % (early_stopping.fname))
    model.load_state_dict(early_stopping.load_checkpoint())

# set early_stopping
if opt.early_stopping > 0:
    early_stopping = EarlyStopping(patience=opt.early_stopping, verbose=False)
    print("Model is saving to: %s" % (early_stopping.fname))

if opt.no_tensorboard is False:
    tb_writer = SummaryWriter(
        comment=f"-dataset_{opt.dataset}-type_{opt.type}"
    )

nclass: 7	nfeat: 1433


In [71]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


# define the training function.
def train(epoch, train_adj, train_fea, idx_train, val_adj=None, val_fea=None):
    if val_adj is None:
        val_adj = train_adj
        val_fea = train_fea

    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(train_fea, train_adj)
    # special for reddit
    if sampler.learning_type == "inductive":
        loss_train = F.nll_loss(output, labels[idx_train])
        acc_train = accuracy(output, labels[idx_train])
    else:
        loss_train = F.nll_loss(output[idx_train], labels[idx_train])
        acc_train = accuracy(output[idx_train], labels[idx_train])

    loss_train.backward()
    optimizer.step()
    train_t = time.time() - t
    val_t = time.time()
    # We can not apply the fastmode for the reddit dataset.
    # if sampler.learning_type == "inductive" or not opt.fastmode:

    if opt.early_stopping > 0 and sampler.dataset != "reddit":
        loss_val = F.nll_loss(output[idx_val], labels[idx_val]).item()
        early_stopping(loss_val, model)

    if not opt.fastmode:
        #    # Evaluate validation set performance separately,
        #    # deactivates dropout during validation run.
        model.eval()
        output = model(val_fea, val_adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val]).item()
        acc_val = accuracy(output[idx_val], labels[idx_val]).item()
        if sampler.dataset == "reddit":
            early_stopping(loss_val, model)
    else:
        loss_val = 0
        acc_val = 0

    if opt.lradjust:
        scheduler.step()

    val_t = time.time() - val_t
    return (loss_train.item(), acc_train.item(), loss_val, acc_val, get_lr(optimizer), train_t, val_t)


def test(test_adj, test_fea):
    model.eval()
    output = model(test_fea, test_adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    auc_test = roc_auc_compute_fn(output[idx_test], labels[idx_test])
    if opt.debug:
        print("Test set results:",
              "loss= {:.4f}".format(loss_test.item()),
              "auc= {:.4f}".format(auc_test),
              "accuracy= {:.4f}".format(acc_test.item()))
        print("accuracy=%.5f" % (acc_test.item()))
    return (loss_test.item(), acc_test.item())


In [72]:
# Train model
t_total = time.time()
loss_train = np.zeros((opt.epochs,))
acc_train = np.zeros((opt.epochs,))
loss_val = np.zeros((opt.epochs,))
acc_val = np.zeros((opt.epochs,))

sampling_t = 0

for epoch in range(opt.epochs):
    input_idx_train = idx_train
    sampling_t = time.time()
    # no sampling
    # randomedge sampling if opt.sampling_percent >= 1.0, it behaves the same as stub_sampler.
    (train_adj, train_fea) = sampler.randomedge_sampler(percent=opt.sampling_percent, normalization=opt.normalization,
                                                        cuda=opt.cuda)
    if opt.mixmode:
        train_adj = train_adj.cuda()

    sampling_t = time.time() - sampling_t
    
    # The validation set is controlled by idx_val
    # if sampler.learning_type == "transductive":
    if False:
        outputs = train(epoch, train_adj, train_fea, input_idx_train)
    else:
        (val_adj, val_fea) = sampler.get_test_set(normalization=opt.normalization, cuda=opt.cuda)
        if opt.mixmode:
            val_adj = val_adj.cuda()
        outputs = train(epoch, train_adj, train_fea, input_idx_train, val_adj, val_fea)

    if opt.debug and epoch % 1 == 0:
        print('Epoch: {:04d}'.format(epoch + 1),
              'loss_train: {:.4f}'.format(outputs[0]),
              'acc_train: {:.4f}'.format(outputs[1]),
              'loss_val: {:.4f}'.format(outputs[2]),
              'acc_val: {:.4f}'.format(outputs[3]),
              'cur_lr: {:.5f}'.format(outputs[4]),
              's_time: {:.4f}s'.format(sampling_t),
              't_time: {:.4f}s'.format(outputs[5]),
              'v_time: {:.4f}s'.format(outputs[6]))
    
    if opt.no_tensorboard is False:
        tb_writer.add_scalars('Loss', {'train': outputs[0], 'val': outputs[2]}, epoch)
        tb_writer.add_scalars('Accuracy', {'train': outputs[1], 'val': outputs[3]}, epoch)
        tb_writer.add_scalar('lr', outputs[4], epoch)
        tb_writer.add_scalars('Time', {'train': outputs[5], 'val': outputs[6]}, epoch)
        

    loss_train[epoch], acc_train[epoch], loss_val[epoch], acc_val[epoch] = outputs[0], outputs[1], outputs[2], outputs[
        3]

    if opt.early_stopping > 0 and early_stopping.early_stop:
        print("Early stopping.")
        model.load_state_dict(early_stopping.load_checkpoint())
        break

if opt.early_stopping > 0:
    model.load_state_dict(early_stopping.load_checkpoint())

if opt.debug:
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 1.9663 acc_train: 0.2343 loss_val: 0.0000 acc_val: 0.0000 cur_lr: 0.02000 s_time: 0.0000s t_time: 0.3150s v_time: 0.0000s
Epoch: 0002 loss_train: 3.5028 acc_train: 0.2914 loss_val: 0.0000 acc_val: 0.0000 cur_lr: 0.02000 s_time: 0.0000s t_time: 0.3050s v_time: 0.0000s
Epoch: 0003 loss_train: 2.5950 acc_train: 0.1714 loss_val: 0.0000 acc_val: 0.0000 cur_lr: 0.02000 s_time: 0.0000s t_time: 0.3160s v_time: 0.0000s
Epoch: 0004 loss_train: 2.1288 acc_train: 0.2161 loss_val: 0.0000 acc_val: 0.0000 cur_lr: 0.02000 s_time: 0.0000s t_time: 0.4230s v_time: 0.0000s
Epoch: 0005 loss_train: 1.8107 acc_train: 0.3137 loss_val: 0.0000 acc_val: 0.0000 cur_lr: 0.02000 s_time: 0.0000s t_time: 0.4080s v_time: 0.0000s
Epoch: 0006 loss_train: 1.9231 acc_train: 0.2914 loss_val: 0.0000 acc_val: 0.0000 cur_lr: 0.02000 s_time: 0.0000s t_time: 0.3190s v_time: 0.0000s
Epoch: 0007 loss_train: 1.7579 acc_train: 0.2939 loss_val: 0.0000 acc_val: 0.0000 cur_lr: 0.02000 s_time: 0.0010s t_time: 0.

In [73]:
(test_adj, test_fea) = sampler.get_test_set(normalization=opt.normalization, cuda=opt.cuda)
if opt.mixmode:
    test_adj = test_adj.cuda()
(loss_test, acc_test) = test(test_adj, test_fea)

print("%s\t%s\t%s\t%s\t%s\t\t%s" % ("loss_train", "loss_val", "loss_test", "acc_train", "acc_val", "acc_test"))
print("%.6f\t%.6f\t%.6f\t%.6f\t%.6f\t%.6f" % (loss_train[-1], loss_val[-1], loss_test, acc_train[-1], acc_val[-1], acc_test))

Test set results: loss= 0.5295 auc= 0.9747 accuracy= 0.8530
accuracy=0.85300
loss_train	loss_val	loss_test	acc_train	acc_val		acc_test
0.205118	0.000000	0.529509	0.934603	0.000000	0.853000
